In [ ]:
!apt update && apt install -y openslide-tools
!pip install openslide-python
!pip install tiatoolbox
!pip install histomicstk --find-links https://girder.github.io/large_image_wheels
!pip install tissueloc==2.1.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/My Drive/2_PHDWork/1_Objective-I/'

/content/drive/My Drive/2_PHDWork/1_Objective-I


In [ ]:
from openslide import OpenSlide
#Ostu tissue mask generator.
from tiatoolbox.tools.tissuemask import OtsuTissueMasker
# Histomics tissue mask generator.
from histomicstk.saliency.tissue_detection import get_tissue_mask
# TissueLoc tissue mask generator.
import tissueloc as tl
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import time
import math
import csv

In [ ]:
"""
The global variable used for selecting the basic mask generation method.
"""
segmentation_method = 'Otsu'

In [ ]:
#Source @ https://github.com/mhdadk/sutherland-hodgman/blob/main/SH.py
# POINTS NEED TO BE PRESENTED CLOCKWISE OR ELSE THIS WONT WORK
import warnings
class PolygonClipper:

    def __init__(self,warn_if_empty=True):
        self.warn_if_empty = warn_if_empty

    def is_inside(self,p1,p2,q):
        R = (p2[0] - p1[0]) * (q[1] - p1[1]) - (p2[1] - p1[1]) * (q[0] - p1[0])
        if R <= 0:
            return True
        else:
            return False

    def compute_intersection(self,p1,p2,p3,p4):

        """
        given points p1 and p2 on line L1, compute the equation of L1 in the
        format of y = m1 * x + b1. Also, given points p3 and p4 on line L2,
        compute the equation of L2 in the format of y = m2 * x + b2.

        To compute the point of intersection of the two lines, equate
        the two line equations together

        m1 * x + b1 = m2 * x + b2

        and solve for x. Once x is obtained, substitute it into one of the
        equations to obtain the value of y.

        if one of the lines is vertical, then the x-coordinate of the point of
        intersection will be the x-coordinate of the vertical line. Note that
        there is no need to check if both lines are vertical (parallel), since
        this function is only called if we know that the lines intersect.
        """

        # if first line is vertical
        if p2[0] - p1[0] == 0:
            x = p1[0]

            # slope and intercept of second line
            m2 = (p4[1] - p3[1]) / (p4[0] - p3[0])
            b2 = p3[1] - m2 * p3[0]

            # y-coordinate of intersection
            y = m2 * x + b2

        # if second line is vertical
        elif p4[0] - p3[0] == 0:
            x = p3[0]

            # slope and intercept of first line
            m1 = (p2[1] - p1[1]) / (p2[0] - p1[0])
            b1 = p1[1] - m1 * p1[0]

            # y-coordinate of intersection
            y = m1 * x + b1

        # if neither line is vertical
        else:
            m1 = (p2[1] - p1[1]) / (p2[0] - p1[0])
            b1 = p1[1] - m1 * p1[0]

            # slope and intercept of second line
            m2 = (p4[1] - p3[1]) / (p4[0] - p3[0])
            b2 = p3[1] - m2 * p3[0]

            # x-coordinate of intersection
            x = (b2 - b1) / (m1 - m2)

            # y-coordinate of intersection
            y = m1 * x + b1

        intersection = (x,y)

        return intersection

    def clip(self,subject_polygon,clipping_polygon):

        final_polygon = subject_polygon.copy()

        for i in range(len(clipping_polygon)):

            # stores the vertices of the next iteration of the clipping procedure
            next_polygon = final_polygon.copy()

            # stores the vertices of the final clipped polygon
            final_polygon = []

            # these two vertices define a line segment (edge) in the clipping
            # polygon. It is assumed that indices wrap around, such that if
            # i = 1, then i - 1 = K.
            c_edge_start = clipping_polygon[i - 1]
            c_edge_end = clipping_polygon[i]

            for j in range(len(next_polygon)):

                # these two vertices define a line segment (edge) in the subject
                # polygon
                s_edge_start = next_polygon[j - 1]
                s_edge_end = next_polygon[j]

                if self.is_inside(c_edge_start,c_edge_end,s_edge_end):
                    if not self.is_inside(c_edge_start,c_edge_end,s_edge_start):
                        intersection = self.compute_intersection(s_edge_start,s_edge_end,c_edge_start,c_edge_end)
                        final_polygon.append(intersection)
                    final_polygon.append(tuple(s_edge_end))
                elif self.is_inside(c_edge_start,c_edge_end,s_edge_start):
                    intersection = self.compute_intersection(s_edge_start,s_edge_end,c_edge_start,c_edge_end)
                    final_polygon.append(intersection)

        return np.asarray(final_polygon)

    def __call__(self,A,B):
        clipped_polygon = self.clip(A,B)
        if len(clipped_polygon) == 0 and self.warn_if_empty:
            warnings.warn("No intersections found. Are you sure your \
                          polygon coordinates are in clockwise order?")

        return clipped_polygon

In [ ]:
"""
The path and name of the log file to which comparison data is writted.

Note: Set it to appropriate filepath and set the 'enable_log' to True for
      writing comparison data.
"""
log_file_name = './log_data_patch_wise_details.csv'

"""
Writes the comparison data to a csv file.

Parameters:
     log_data: the comparison data to be written.

Returns:
     None
"""
def log_comparison_data(log_data):
  fields = ['dataset','f_name','seg_method','l_width','l_height','h_width',
            'h_height','l_cntr_count','h_cntr_count','patch_count',
            'max_patch_width','max_patch_height','avg_cntr_trans_time',
            'avg_msk_gen_time','avg_mth_msk_gen_time','avg_accuracy',
            'avg_dice_p','avg_jaccard_p', 'avg_dice_n','avg_jaccard_n']

  # Creating a log file if it does not exist.
  if(not os.path.isfile(log_file_name)):
    with open(log_file_name, 'w') as csvfile:
      # creating a csv writer object
      csvwriter = csv.writer(csvfile)
      # writing the fields
      csvwriter.writerow(fields)

  # Writing comparison data to the file.
  try:
    with open(log_file_name,"a") as csvFile:
      writer = csv.DictWriter(csvFile, fieldnames=fields)
      writer.writerow(log_data)
  except IOError:
    print("Writing of Training data failed")

  return

In [ ]:
"""
Accepts the image and generates ist tissue mask by the proposed method.

Parameters:
     thum_nail: numpy array denoting the image for which mask has to be
                generated.

     method:    the method to be used for generation.

Returns:
     bin_mask:  the tissue mask.
"""
def get_image_tissue_mask(thum_nail,method='Otsu'):
  bin_mask = None

  if(method == "Otsu"):
    t_size = thum_nail.shape
    thum_nail = np.reshape(thum_nail,(1,t_size[0],t_size[1],t_size[2]))
    masker = OtsuTissueMasker()
    masker.fit(thum_nail)
    masks = masker.transform(thum_nail)
    true_index = np.asarray(masks[0] == True).nonzero()
    bin_mask = np.zeros(masks[0].shape,dtype=np.uint8)
    bin_mask[true_index] = 255
    thum_nail = np.reshape(thum_nail,(t_size[0],t_size[1],t_size[2]))

  elif(method == "Histomics"):
    thm_nail = cv.cvtColor(thum_nail,cv.COLOR_RGBA2RGB)
    bin_mask, _ = get_tissue_mask(thm_nail, deconvolve_first=False,
                                  n_thresholding_steps=1, sigma=0., min_size=30)
    true_index = np.asarray(bin_mask > 0).nonzero()
    bin_mask[:,:] = 0
    bin_mask[true_index] = 255
    bin_mask = bin_mask.astype(np.uint8)

  elif(method == 'TissueLoc'):
    thm_nail = cv.cvtColor(thum_nail,cv.COLOR_RGBA2RGB)
    thm_nail = tl.rgb2gray(thm_nail)
    thm_nail = tl.thresh_slide(thm_nail, 0.80, sigma=13)
    thm_nail = tl.fill_tissue_holes(thm_nail)
    thm_nail = tl.remove_small_tissue(thm_nail, 1)
    cnts = tl.find_tissue_cnts(thm_nail)
    bin_mask = np.zeros(thm_nail.shape,dtype=np.uint8)
    bin_mask = cv.fillPoly(bin_mask, cnts, (255))

  else:
    print('Error: No segmentation method specified.')

  return bin_mask

In [ ]:
"""
Reads the thumbnail image of the given wsi file, performs tissue segmentation and
converts the mask into boundary coordinates.

Parameters:
     wsifilename: the name of the wsi file with path for which the prediction
                  has to be performed.

     display:     if true displays the processed images.

Returns:
     contour:     the boundary co-ordinates.
     shp:         the row and col count of the thumbnail.
"""
def get_thumbnail_cordinates(wsifilename,display=False):
  wsi = OpenSlide(wsifilename)
  level = wsi.level_count-1
  thum_nail = np.array(wsi.read_region((0,0),level,wsi.level_dimensions[level]))
  shp = (thum_nail.shape[0],thum_nail.shape[1])

  bin_mask = get_image_tissue_mask(thum_nail,method=segmentation_method)
  contours, hierarchy = cv.findContours(bin_mask, cv.RETR_EXTERNAL,
                                        cv.CHAIN_APPROX_NONE)

  if (display):
    print("----------------------WSI Details----------------------------------")
    #print('Name:             ',name)
    print('Level Count:      ',wsi.level_count)
    print('Dimensions:       ',wsi.level_dimensions)
    print('Downscale Factors:',wsi.level_downsamples)
    plt.figure(figsize=(9,6))
    plt.subplot(1,3,1)
    plt.title('Thumbnail')
    plt.imshow(thum_nail[0])
    # plt.subplot(1,5,2)
    # plt.title("Mask")
    # plt.imshow(masks[0])
    plt.subplot(1,3,2)
    plt.title("Binary Mask")
    plt.imshow(bin_mask)

    img = np.zeros(bin_mask.shape,dtype=np.uint8)
    cv.drawContours(img, contours, -1, 255, 3)
    plt.subplot(1,3,3)
    plt.title("Boundary Generated")
    plt.imshow(img)
    plt.show()
    print("-------------------------------------------------------------------")

  return (shp,contours)

In [ ]:
"""
Reads the shape and the scaling factor of high resolution image at a specified
level of the given wsi file.

Parameters:
     wsifilename: the name of the wsi file with path for which the prediction
                  has to be performed.

     level:       the high resolution level from which the details has to be
                  read.

     display:     if true displays the processed images.

Returns:
     hrs_shp:     the row and col count of the high resolution image.
     s_r:         the row scale factor.
     s_c:         the col scale factor.
"""
def get_higrs_image_details(wsifilename,level=1,display=False):
  wsi = OpenSlide(wsifilename)
  thum_level = wsi.level_count-1

  thumb_shp = (wsi.level_dimensions[thum_level][1],
               wsi.level_dimensions[thum_level][0])
  hrs_shp = (wsi.level_dimensions[level][1],wsi.level_dimensions[level][0])

  s_r = hrs_shp[0] / thumb_shp[0]
  s_c = hrs_shp[1] / thumb_shp[1]

  if(display):
    print('High Resolution Image Dim: ',hrs_shp)
    print('Low Resolution Image Dim : ',thumb_shp)
    print('Row Scale Factor         : ',s_r)
    print('Col Scale Factor         : ',s_c)

  return (hrs_shp,s_r,s_c)

In [ ]:
"""
Transform the contour points of the low resolution image to high resolutio based
on the developed equation.

Parameters:
     transform_request: the details required for the transformation equation.

Returns:
     transform_points:  the transformed points.

     elapsed_time:      the time is seconds required for generating the new
                        points
"""
def transform_contour_points(transform_request):
  tot = len(transform_request)
  transform_points = []

  for i in range(tot):
    start_time = time.process_time()
    item = transform_request[i]
    s_cent = item[0]  #The centroid of the low resolution image.
    l_cent = item[1]  #The centroid of the high resolution image.
    cot_points = item[2]
    s_r = item[3][0]  #The row downscale factor.
    s_c = item[3][1]  #The col downscale factor.
    new_points = []

    for pnts in cot_points:
      # Step-1: Converting the points to float for performing mathematical operation.
      wrk_pnts = pnts.astype(np.float32)

      # Step-2: Shifting the points to align centroid with the origin.
      wrk_pnts[:,:,0] = wrk_pnts[:,:,0] - s_cent[0] #Row Co-ordinates.
      wrk_pnts[:,:,1] = wrk_pnts[:,:,1] - s_cent[1] #Col Co-ordinates.

      # Step-3: Determining the negative points for use in the future.
      xa_posi_index = np.asarray(wrk_pnts[:,:,0] < 0).nonzero()
      ya_posi_index = np.asarray(wrk_pnts[:,:,1] < 0).nonzero()

      # Step-4: Squaring element-wise and multiplying it with the square of the scale.
      wrk_pnts = np.square(wrk_pnts)
      wrk_pnts[:,:,0] = wrk_pnts[:,:,0] * s_r * s_r
      wrk_pnts[:,:,1] = wrk_pnts[:,:,1] * s_c * s_c

      #Step-5: Taking square root
      wrk_pnts = np.sqrt(wrk_pnts)

      #Step-6: Re-assigning negative sign
      wrk_pnts[xa_posi_index[0],xa_posi_index[1],0] = wrk_pnts[xa_posi_index[0],xa_posi_index[1],0] * -1
      wrk_pnts[ya_posi_index[0],ya_posi_index[1],1] = wrk_pnts[ya_posi_index[0],ya_posi_index[1],1] * -1

      #Step-7: Re-shifting
      wrk_pnts[:,:,0] = wrk_pnts[:,:,0] + l_cent[0]
      wrk_pnts[:,:,1] = wrk_pnts[:,:,1] + l_cent[1]

      #Step-8: Finding the floor
      wrk_pnts = (np.floor(wrk_pnts)).astype(np.int32)

      new_points.append(wrk_pnts)

    transform_points.append(tuple(new_points))
    end_time = time.process_time()
    elapsed_time = (end_time - start_time)
  return (transform_points,elapsed_time)

In [ ]:
"""
Generates the patch masks based on the given contour and patch co-ordinates.

Parameters:
     contours: the contour points of the entire mask image.

     row_top:  the top row co-ordinate of the patch.

     col_top:  the top col co-ordinate of the patch.

     height:   height of the patch.

     width:    width of the patch.

     display:  display flag. Set it True if display of information is required.

Returns:
     masks:          the generated mask of the patch.

     contr_gen_time: the time required to estimate the contour points of the
                     patch from the overal image contour.

     mask_gen_time:  the time required to create a mask from the generated
                     contours.

Functioning:
    The method makes use of the polygon clipping concept for determining the
    patch contours.

"""
def generate_mask_by_approximation(contours,row_top,col_top,height,width,
                              display = False):
  masks = np.zeros((height,width),dtype=np.uint8)
  cip_window = [(col_top,row_top),(col_top,(row_top+height - 1)),
                ((col_top + width - 1),(row_top + height - 1)),
                ((col_top + width - 1),row_top)
               ]

  # Commented to measure the exact time of computation, Un-comment for debugging
  # if(display):
  #   print("Clip Window Co-ordinates: ")
  #   print(cip_window)

  clip = PolygonClipper(warn_if_empty=False)
  clipped_cntr = []

  start_time = time.process_time()
  for cnt in contours:
    rs_c = np.reshape(cnt,(cnt.shape[0],2))
    wk_c = np.copy(rs_c)
    wk_c[:,0] = rs_c[:,1]
    wk_c[:,1] = rs_c[:,0]
    w_c = clip(wk_c,cip_window)

    if(w_c.shape[0] >= 1):
      n_c = np.copy(w_c)
      n_c[:,0] = w_c[:,1]- row_top
      n_c[:,1] = w_c[:,0] - col_top
      n_c = n_c.astype(np.int32)

      # To compensate where the entire image block needs to be filled.
      n_c[(n_c[:,0] == 0),0] = 1
      n_c[(n_c[:,1] == 0),1] = 1

      # Commented to measure the exact time of computation, Un-comment for
      # debugging.
      # if(display):
      #   print("Row Top: ",row_top)
      #   print("Col Top: ",col_top)
      #   print("Original Shape: ",rs_c.shape)
      #   print(rs_c)
      #   print("Clipped: ",w_c.shape)
      #   print(w_c)
      #   print('Transformed: ',n_c.shape)
      #   print(n_c)

      n_c = np.reshape(n_c,(n_c.shape[0],1,2))
      clipped_cntr.append((n_c.astype(np.int32)))
  end_time = time.process_time()
  contr_gen_time = (end_time - start_time)
  start_time = time.process_time()
  if(len(clipped_cntr) >= 1):
    masks = cv.fillPoly(masks, clipped_cntr, (255))
  end_time = time.process_time()
  mask_gen_time = (end_time - start_time)

  return (masks,contr_gen_time,mask_gen_time)

In [ ]:
"""
Generates the patch-wise masks of the level-0 image from the transformed
contour of the tissue mask of the thumnail image.

Parameters:
     wsifilename: the name of the WSI file.

     old_points:  the contour points of the tiisue mask of the thumbnail image.

     new_points:  the transformed contours of the tissue mask of the thumbnail
                  image.

     method_time: the time taken for generating the transformed contours.

     log_data:    the dictionary to which the comparison data are to be
                  appended.

     pt_patch_dt: set to True if the patch-wise details is to be printed.

     patch_row_size: the row size of the patch to be extracted.

     patch_col_size: the col size of the patch to be extracted.

     draw_figure: set to True if the patch-wise figure is to be drawn. Please
                  note setting it to True would consume huge memory.

Returns:
     log_data:    the measurement data generated as part of the comaprison.

"""
def perform_patch_wise_mask_generation(wsifilename, old_points,new_points,
                                       method_time, log_data,
                                       pt_patch_dt = False,
                                       patch_row_size = 3000,
                                       patch_col_size = 3000, draw_figure=False):
  wsi = OpenSlide(wsifilename)
  hrs_shp = (wsi.level_dimensions[0][0],wsi.level_dimensions[0][1])

  row_patches = hrs_shp[1] // patch_row_size
  if((hrs_shp[1] % patch_row_size) > 0):
    row_patches += 1

  col_patches = hrs_shp[0] // patch_col_size
  if((hrs_shp[0] % patch_col_size) > 0):
    col_patches += 1

  print("\tProcessing WSI File          : ",wsifilename)
  print("\tDimension (width and height) : ",hrs_shp)
  print("\tTotal Patches                : ",row_patches*col_patches)

  if(draw_figure):
    # wd = col_patches * 4
    # hg = row_patches * 3
    wd = 15
    hg = 13
    fig1 = plt.figure(layout="constrained",figsize=(wd,hg))
    fig2 = plt.figure(layout="constrained",figsize=(wd,hg))
    fig3 = plt.figure(layout="constrained",figsize=(wd,hg))
    fig_index = 1

  t_cntr_trans_time = 0.0
  t_msk_gen_time = 0.0
  t_mth_msk_gen_time = 0.0
  t_accuracy = 0.0
  t_dice_p = 0.0
  t_jaccard_p = 0.0
  t_dice_n = 0.0
  t_jaccard_n = 0.0

  patch_details = []
  # Generating the patch of level 0 image patches.
  x = 0
  for i in range(row_patches): # Row Loop
    # Determing the height of the patch to be read.
    if((x + patch_row_size - 1) > (hrs_shp[1] - 1)):
      height = (hrs_shp[1] - x )
    else:
      height = patch_row_size
    y = 0
    for j in range(col_patches): #Column Loop
      # Determing the width of the patch to be read.
      if((y + patch_col_size - 1) > (hrs_shp[0] - 1)):
        width = hrs_shp[0] - y
      else:
        width = patch_col_size

      print("Processing Patch at location: (",y,x,
            ") and dimension (width and height): (", width,height,")")

      patch = np.array(wsi.read_region((y,x),0,(width,height)))
      mask,contr_gen_time,mask_gen_time = generate_mask_by_approximation(
                                                    new_points,y,x,height,width)
        # plt.figure(figsize=(8,6))
        # plt.subplot(1,2,1)
        # plt.imshow(patch)
        # plt.subplot(1,2,2)
        # plt.imshow(mask)

      start_time = time.process_time()
      bin_mask = get_image_tissue_mask(patch,method=segmentation_method)
      end_time = time.process_time()
      elapsed_time = (end_time - start_time)

      accuracy,dice_p,jaccard_p,dice_n,jaccard_n = measure_accuracy(bin_mask,
                                                                    mask)

      t_cntr_trans_time += contr_gen_time
      t_msk_gen_time += mask_gen_time
      t_mth_msk_gen_time += elapsed_time
      t_accuracy += accuracy
      t_dice_p += dice_p
      t_jaccard_p += jaccard_p
      t_dice_n += dice_n
      t_jaccard_n += jaccard_n

      data = (mask.shape,elapsed_time,contr_gen_time,mask_gen_time,accuracy,
              dice_p,jaccard_p,dice_n,jaccard_n)
      patch_details.append(data)

      if(draw_figure):
        ax = fig1.add_subplot(row_patches,col_patches,fig_index)
        ax.imshow(patch)
        ax1 = fig2.add_subplot(row_patches,col_patches,fig_index)
        ax1.imshow(bin_mask)
        ax3 = fig3.add_subplot(row_patches,col_patches,fig_index)
        ax3.imshow(mask)
        fig_index += 1

      y += width
    x += height

  total_patches = row_patches*col_patches
  t_cntr_trans_time = round((t_cntr_trans_time / total_patches),6)
  t_msk_gen_time = round((t_msk_gen_time/total_patches),6)
  t_mth_msk_gen_time = round((t_mth_msk_gen_time/total_patches),6)
  t_accuracy = round((t_accuracy/total_patches),2)
  t_dice_p = round((t_dice_p/total_patches),2)
  t_jaccard_p = round((t_jaccard_p/total_patches),2)
  t_dice_n = round((t_dice_n/total_patches),2)
  t_jaccard_n = round((t_jaccard_n/total_patches),2)

  log_data['l_width'] = wsi.level_dimensions[wsi.level_count-1][0]
  log_data['l_height'] = wsi.level_dimensions[wsi.level_count-1][1]
  log_data['h_width'] = hrs_shp[0]
  log_data['h_height'] = hrs_shp[0]
  log_data['l_cntr_count'] = len(old_points)
  log_data['h_cntr_count'] = len(new_points)
  log_data['patch_count'] = total_patches
  log_data['max_patch_width'] = patch_col_size
  log_data['max_patch_height'] = patch_row_size
  log_data['avg_cntr_trans_time'] = t_cntr_trans_time * 10**3
  log_data['avg_msk_gen_time'] = t_msk_gen_time * 10**3
  log_data['avg_mth_msk_gen_time'] = t_mth_msk_gen_time * 10**3
  log_data['avg_accuracy'] = t_accuracy
  log_data['avg_dice_p'] = t_dice_p
  log_data['avg_jaccard_p'] = t_jaccard_p
  log_data['avg_dice_n'] = t_dice_n
  log_data['avg_jaccard_n'] = t_jaccard_n

  print("----------------------Transformation Details-----------------------  ")
  print("Width and Height of Thumnail Image          : ",
                                        wsi.level_dimensions[wsi.level_count-1])
  print("Width and Height of 0 level Image           : ",hrs_shp)
  print("Low resolution Image Contour Length         : ",len(old_points))
  print("Transform Contour Length                    : ",len(new_points))
  print("Level 0 Images as Patches of (rows and cols): (",
                                                    row_patches,col_patches,")")
  print("Patch maximum Dimension (row and col)       : (",patch_row_size,
                                                             patch_col_size,")")
  print("Average Time Taken by "+segmentation_method+" Segmentation     : ",
                                                t_mth_msk_gen_time * 10**3,"ms")
  print("Average Time Taken for contour generation   : ",
                                                 t_cntr_trans_time * 10**3,"ms")
  print("Average Time Taken for mask generation      : ",
                                                    t_msk_gen_time * 10**3,"ms")
  print("Average Reconstruction Accuracy             : ",t_accuracy)
  print("Average Dice Coefficients (Tissue Pixels)   : ",t_dice_p)
  print("Average Jaccard's Coefficients (Tisse)      : ",t_jaccard_p)
  print("Average Dice Coefficients (Non-Tissue)      : ",t_dice_n)
  print("Average Jaccard's Coefficients (Non-Tisse)  : ",t_jaccard_n)

  if(pt_patch_dt):
    print("Patch Details are as follows                : ................")
    data_index = 0
    for i in range(row_patches):
      for j in range(col_patches):
        data = patch_details[data_index]
        data_index += 1
        print("\t-------------------------------------------------")
        print("\tPatch Index                         :  (",i,j,")")
        print("\tPatch shape                         : ",data[0])
        print("\tTime Taken by "+segmentation_method+" Segmentation     : ",
                                                        data[1] * 10**3,"ms")
        print("\tTime Taken for contour generation   : ",data[2] * 10**3,"ms")
        print("\tTime Taken for mask generation      : ",data[3] * 10**3,"ms")
        print("\tReconstruction accuarcy             : ",data[4])
        print("\tDice Coefficients (Tissue Pixels)   : ",data[5])
        print("\tJaccard's Coefficients (Tisse)      : ",data[6])
        print("\tDice Coefficients (Non-Tissue)      : ",data[7])
        print("\tJaccard's Coefficients (Non-Tisse)  : ",data[8])

  if(draw_figure):
    path = "./1_Results/"
    fig1.savefig(path + 'Original.pdf', format='pdf')
    fig3.savefig(path + segmentation_method + '.pdf', format='pdf')
    fig1.show()
    # fig2.show()
    fig3.show()
  print("-------------------------------------------------------------------  ")

  return log_data

In [ ]:
"""
Performs a pixel to pixel matching between the given masks and returns the
accuracy, Dice's and Jaccard's coefficient.

Parameters:
     org:     the original labels.

     img2:    the predicted labels.

Returns:
     accuracy:    the percentage of pixels that matches.

     dice_p:      the Dice's coefficients for tissue pixels.

     jaccard_p:   the Jaccard's coefficients for tissue pixels.

     dice_n:      the Dice's coefficients for non-tissue pixels.

     jaccard_n:   the Jaccard's coefficients for non-tissue pixels.

"""
def measure_accuracy(org,pred):
  # Converting the values into 0 and 1.
  i1 = org // 255
  i2 = pred // 255

  # Calculating the TP, TN, FP and FN
  TP = np.sum((i1 * i2))
  TN = np.sum((1 - i1) * (1 - i2))
  FP = np.sum((((i2 - i1) + 1)//2))
  FN = np.sum((((i1 - i2) + 1)//2))

  accuracy = round(((TP + TN)/(TP + TN + FP + FN)),2) * 100
  dice_p = round(((2 * TP)/((2 * TP) + FP + FN)),2)
  jaccard_p = round((TP /(TP + FP + FN)),2)
  dice_n = round(((2 * TN)/((2 * TN) + FP + FN)),2)
  jaccard_n = round((TN /(TN + FP + FN)),2)

  if(math.isnan(accuracy)):
    accuracy = 0
  if(math.isnan(dice_p)):
    dice_p = 0
  if(math.isnan(jaccard_p)):
    jaccard_p = 0
  if(math.isnan(dice_n)):
    dice_n = 0
  if(math.isnan(jaccard_n)):
    jaccard_n = 0

  return (accuracy,dice_p,jaccard_p,dice_n,jaccard_n)

In [ ]:
"""
Performs the co-ordinate prediction of the tissue area from the thumbnail
image of the given level image.

Parameters:
     wsifilename: the name of the wsi file with path for which the prediction
                  has to be performed.

 Returns:
    thumb_nail_cor: the contour co-ordinates for the low resolution image.

    new_points[0]:  the contour co-ordinates for the high resolution image.

    method_time:    the time taken by the proposed method to create the points.e
"""
def predict_cordinates(wsifilename):
  trans_req = []
  t_shape, thumb_nail_cor = get_thumbnail_cordinates(wsifilename,display=False)
  h_shape, s_r, s_c = get_higrs_image_details(wsifilename,level=0,display=False)
  l_cent = (h_shape[0]/2,h_shape[1]/2)
  s_cent = (t_shape[0]/2,t_shape[1]/2)
  sample = [s_cent,l_cent,thumb_nail_cor,(s_r,s_c)]
  trans_req.append(sample)
  new_points,method_time = transform_contour_points(trans_req)

  return (thumb_nail_cor,new_points[0],method_time)


In [ ]:
"""
The entry level function to be invoked to evaluate the working of the proposed
method.

Parameters:
     filename: the wsi file name on which the method has to be applied.

     dataset:  the identifier to recognize the dataset used in loging data.

     draw_figure: set to True if the patch-wise figure is to be drawn. Please
                  not setting it to True would consume huge memory.

     enable_log: the flag variable for controlling data log. Set it to True to
                 log comparison data

 Returns:
     None

 Functioning:
    Step-1 : If a directory path is passed, the files within it is read
             one-by-one for processing. Ensure that the directory contains only
             wsi files.

    Step-2 : The method 'predict_cordinates' is invoked on each wsi file
             which generates the contour points for the thumbnail and
             level-0 image.

    Step-3 : Once the contour points are available, the method
             'perform_patch_wise_mask_generation' is invoked to generate the
             patch-wise masks and compare it with Otsu thresholding based
             segmentation.
"""
def main_fun(path,dataset,draw_figure=False,enable_log = True):

  if(os.path.isdir(path)):
    files_names = os.listdir(path)

    for file in files_names:
      wsiname = path + file
      log_data = {}
      log_data['dataset'] = dataset
      log_data['f_name'] = file
      log_data['seg_method'] = segmentation_method

      low_res_cor,new_points,method_time = predict_cordinates(wsiname)
      log_data = perform_patch_wise_mask_generation(wsiname,low_res_cor,
                                                    new_points,method_time,
                                                    log_data,pt_patch_dt=False,
                                                    draw_figure=draw_figure)

      if(enable_log):
        log_comparison_data(log_data)

  else:
    wsiname = path
    name = os.path.basename(path)
    log_data = {}
    log_data['dataset'] = dataset
    log_data['f_name'] = name
    log_data['seg_method'] = segmentation_method

    low_res_cor,new_points,method_time = predict_cordinates(wsiname)
    log_data = perform_patch_wise_mask_generation(wsiname, low_res_cor,
                                                    new_points, method_time,
                                                    log_data,pt_patch_dt=False,
                                                    draw_figure=draw_figure)

    if(enable_log):
        log_comparison_data(log_data)

  return

# main_fun("./0_Dataset/1_Skin_Cancer/",'Skin_Cancer',draw_figure=False)
# main_fun("./0_Dataset/2_ICIAR2018/",'ICIAR2018',draw_figure=False)
main_fun("./0_Dataset/4_CPTAC-UCEC/C3L-00136-21.svs",'CPTAC-UCEC',
         draw_figure=True,enable_log = False)